# Funda Real Estate Analysis

This notebook demonstrates how to analyze Dutch real estate data using pyfunda.

## Setup

```bash
uv pip install pyfunda pandas matplotlib
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from funda import Funda

plt.rcParams['figure.figsize'] = (10, 6)

## Fetch Listings

In [ ]:
all_listings = []

with Funda() as f:
    for page in range(5):
        results = f.search_listing('amsterdam', page=page)
        if not results:
            break
        all_listings.extend([r.to_dict() for r in results])

print(f"Fetched {len(all_listings)} listings")

In [ ]:
df = pd.DataFrame(all_listings)
cols = ['title', 'city', 'postcode', 'price', 'living_area', 'bedrooms', 'energy_label', 'object_type']
df = df[[c for c in cols if c in df.columns]]
df.head(10)

## Price Analysis

In [ ]:
print("Price Statistics (EUR)")
print("=" * 30)
print(f"Mean:   {df['price'].mean():>12,.0f}")
print(f"Median: {df['price'].median():>12,.0f}")
print(f"Min:    {df['price'].min():>12,.0f}")
print(f"Max:    {df['price'].max():>12,.0f}")

In [ ]:
fig, ax = plt.subplots()
df['price'].hist(bins=20, ax=ax, edgecolor='black')
ax.set_xlabel('Price (EUR)')
ax.set_ylabel('Count')
ax.set_title('Price Distribution in Amsterdam')
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M'))
plt.tight_layout()
plt.show()

## Price per m²

In [ ]:
df_area = df[df['living_area'].notna() & (df['living_area'] > 0)].copy()
df_area['price_per_m2'] = df_area['price'] / df_area['living_area']

print(f"Mean price/m²:   {df_area['price_per_m2'].mean():,.0f}")
print(f"Median price/m²: {df_area['price_per_m2'].median():,.0f}")

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_area['living_area'], df_area['price'], alpha=0.6)
ax.set_xlabel('Living Area (m²)')
ax.set_ylabel('Price (EUR)')
ax.set_title('Price vs Living Area')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M'))
plt.tight_layout()
plt.show()

## Compare Cities

In [ ]:
cities = ['amsterdam', 'rotterdam', 'utrecht', 'den-haag']
city_data = []

with Funda() as f:
    for city in cities:
        results = f.search_listing(city, page=0)
        for r in results:
            city_data.append({'city': city.title(), 'price': r['price']})

df_cities = pd.DataFrame(city_data)
print(f"Fetched {len(df_cities)} listings from {len(cities)} cities")

In [ ]:
city_medians = df_cities.groupby('city')['price'].median().sort_values(ascending=False)

fig, ax = plt.subplots()
city_medians.plot(kind='barh', ax=ax, color='steelblue', edgecolor='black')
ax.set_xlabel('Median Price (EUR)')
ax.set_title('Median House Price by City')
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e3:.0f}K'))
plt.tight_layout()
plt.show()

## Export

In [ ]:
df.to_csv('amsterdam_listings.csv', index=False)
print("Saved to amsterdam_listings.csv")